This notebook implements bag of words on our dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk, random


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os
!ls "/content/drive/My Drive/CS224N DATA"

aapl-20210327-10-q.txt	bagOfWordsGroup.ipynb	goog-20211231-10-k.txt
aapl-20210626-10-q.txt	fb-20210630-10-q.txt	nflx-20210630-10-q.txt
aapl-20210925-10-k.txt	fb-20210930-10-q.txt	nflx-20210930-10-q.txt
amzn-20210630-10-q.txt	fb-20211231-10-k.txt	nflx-20211231-10-k.txt
amzn-20210930-10-q.txt	goog-20210630-10-q.txt	sp500.csv
amzn-20211231-10-k.txt	goog-20210930-10-q.txt	stocks.xlsx


In [ ]:
#run to increase field_size_limit of the csv file
import sys
import csv
maxInt = sys.maxsize
while True:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [ ]:
df = pd.read_csv('/content/drive/My Drive/CS224N DATA/sp500.csv', sep='~', engine='python')

Data is read as: ticker - date - volat3 - volat7 - volat15 - volat30 - Kvolat3 - Kvolat7 - Kvolat15 - Kvolat30 - transcript


In [ ]:
df.head

<bound method NDFrame.head of       ticker  year  quarter   id     t        date    volat3    volat7  \
0          A  2006        2    1   345   5/16/2006 -0.081151 -0.101904   
1          A  2006        3    1   408   8/15/2006  0.154107  0.132567   
2          A  2006        4    1   472  11/14/2006 -0.026609 -0.011770   
3          A  2007        1    1   535   2/16/2007 -0.013323 -0.003462   
4          A  2007        2    1   595   5/15/2007  0.039758  0.035938   
...      ...   ...      ...  ...   ...         ...       ...       ...   
24081    ZTS  2020        2  495  1892    8/6/2020  0.024772  0.049800   
24082    ZTS  2020        3  495  1957   11/6/2020 -0.016482  0.011021   
24083    ZTS  2020        4  495  2024   2/16/2021  0.019983  0.010845   
24084    ZTS  2021        2  495  2143    8/5/2021 -0.013147 -0.013358   
24085    ZTS  2021        3  495  2207   11/4/2021  0.021570  0.023973   

        volat15   volat30   Kvolat3   Kvolat7  Kvolat15  Kvolat30  \
0     -0.106

In [ ]:
print(df.ticker)
print(df.volat3)
print(df.ticker[:10])
print(df.transcript[:10])

0          A
1          A
2          A
3          A
4          A
        ... 
24081    ZTS
24082    ZTS
24083    ZTS
24084    ZTS
24085    ZTS
Name: ticker, Length: 24086, dtype: object
0       -0.081151
1        0.154107
2       -0.026609
3       -0.013323
4        0.039758
           ...   
24081    0.024772
24082   -0.016482
24083    0.019983
24084   -0.013147
24085    0.021570
Name: volat3, Length: 24086, dtype: float64
0    A
1    A
2    A
3    A
4    A
5    A
6    A
7    A
8    A
9    A
Name: ticker, dtype: object
0    Operator: Good day ladies and gentlemen and we...
1    Operator: Good day, ladies and gentlemen, and ...
2    Operator: Good day ladies and gentlemen and we...
3    Operator: Good day, ladies and gentlemen, and ...
4    Operator: Good day, ladies and gentlemen, and ...
5    Operator: Good day ladies and gentlemen, and w...
6    Operator: Good day, ladies and gentlemen, and ...
7    Operator: Good day, ladies and gentlemen, and ...
8    Operator: Good day, ladies an

In [ ]:
from sklearn.model_selection import train_test_split
#train, test = train_test_split(df, test_size=0.2)
validation_years= [2008,2012, 2016, 2020]
validation = df[df['year'].isin(validation_years)]
test_years = [2009, 2013, 2017, 2021]
test = df[df['year'].isin(test_years)]
train = df[~df['year'].isin(validation_years+test_years)]


train[:5]
#test[:5]
#validation[:5]

,ticker,year,quarter,id,t,date,volat3,volat7,volat15,volat30,Kvolat3,Kvolat7,Kvolat15,Kvolat30,transcript
0,A,2006,2,1,345,5/16/2006,-0.081151,-0.101904,-0.106044,-0.126091,-3.817327,-4.195840,-4.412217,-4.328459,Operator: Good day ladies and gentlemen and we...
1,A,2006,3,1,408,8/15/2006,0.154107,0.132567,0.120317,0.092741,-3.549376,-3.742952,-3.841390,-3.761206,"Operator: Good day, ladies and gentlemen, and ..."
2,A,2006,4,1,472,11/14/2006,-0.026609,-0.011770,-0.016055,0.025393,-5.279222,-4.155949,-4.221972,-4.346753,Operator: Good day ladies and gentlemen and we...
3,A,2007,1,1,535,2/16/2007,-0.013323,-0.003462,-0.015071,-0.017384,-4.311276,-4.457299,-4.497159,-4.502518,"Operator: Good day, ladies and gentlemen, and ..."
4,A,2007,2,1,595,5/15/2007,0.039758,0.035938,0.059443,0.079331,-3.994754,-4.120073,-4.050324,-4.277250,"Operator: Good day, ladies and gentlemen, and ..."


In [ ]:
def normalize_list_numpy(list):
    #normalized_list = list / np.linalg.norm(list)
    #return normalized_list
    return (list - list.mean()) / list.std()


Normalize and Come Up with Simple Train Test

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train = train.transcript
X_train[:5]
print(len(X_train))
X_test = test.transcript
X_test[:5]
print(len(X_test))

y_train = train.volat3 
y_test = test.volat3
print(y_train)
print(y_test)



11160
6323
0       -0.081151
1        0.154107
2       -0.026609
3       -0.013323
4        0.039758
           ...   
24075    0.063053
24076    0.007835
24077    0.066116
24078   -0.041637
24079    0.010329
Name: volat3, Length: 11160, dtype: float64
10      -0.158947
11      -0.016771
12       0.047521
13       0.069545
24      -0.053746
           ...   
24069    0.052335
24070   -0.028567
24071    0.079072
24084   -0.013147
24085    0.021570
Name: volat3, Length: 6323, dtype: float64


In [ ]:
y_train = normalize_list_numpy(y_train)
y_test = normalize_list_numpy(y_test)
print(y_train)
print(y_test)

0       -1.458699
1        2.534162
2       -0.533006
3       -0.307508
4        0.593402
           ...   
24075    0.988777
24076    0.051590
24077    1.040761
24078   -0.788066
24079    0.093924
Name: volat3, Length: 11160, dtype: float64
10      -2.778035
11      -0.392942
12       0.685611
13       1.055079
24      -1.013224
           ...   
24069    0.766359
24070   -0.590822
24071    1.214898
24084   -0.332146
24085    0.250258
Name: volat3, Length: 6323, dtype: float64


Text Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf_vec = TfidfVectorizer(min_df = 10, token_pattern = r'[a-zA-Z]+')
X_train_bow = tfidf_vec.fit_transform(X_train) # fit train
X_test_bow = tfidf_vec.transform(X_test) # transform test

In [ ]:
print(X_train_bow.shape)
print(X_test_bow.shape)

(11160, 33742)
(6323, 33742)


1.4. Model Selection and Cross Validation

KNN

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
train_scaled = X_train_bow
test_scaled = X_test_bow

In [ ]:
#Train KNN
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_jobs = -1)
model.fit(train_scaled, y_train)
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mse = mean_squared_error(y_train, model.predict(train_scaled))
mae = mean_absolute_error(y_train, model.predict(train_scaled))
from math import sqrt
print("mse = ",mse," & mae = ",mae," & rmse = ", sqrt(mse))


mse =  0.7990095397298606  & mae =  0.647719887962716  & rmse =  0.8938733353948201


In [ ]:
#Test KNN
test_mse = mean_squared_error(y_test, model.predict(test_scaled))
test_mae = mean_absolute_error(y_test, model.predict(test_scaled))
print("mse = ",test_mse," & mae = ",test_mae," & rmse = ", sqrt(test_mse))

mse =  1.1812440830447335  & mae =  0.7774552419937878  & rmse =  1.0868505339027688


**Regression**

In [ ]:
# Linear Regression

from sklearn.linear_model import LinearRegression
print(1)
modelLR = LinearRegression(n_jobs=-1)
print(2)
modelLR.fit(X_train_bow, y_train)
print(3)

1
2
3


In [ ]:
#Train
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mse = mean_squared_error(y_train, modelLR.predict(X_train_bow))
mae = mean_absolute_error(y_train, modelLR.predict(X_train_bow))

In [ ]:
from math import sqrt
print("mse = ",mse," & mae = ",mae," & rmse = ", sqrt(mse))


mse =  0.00023410885003010253  & mae =  0.0002090034023623976  & rmse =  0.015300616001655049


In [ ]:
# Test Results
test_mse = mean_squared_error(y_test, modelLR.predict(X_test_bow))
test_mae = mean_absolute_error(y_test, modelLR.predict(X_test_bow))
print("mse = ",test_mse," & mae = ",test_mae," & rmse = ", sqrt(test_mse))

mse =  2.276293722073624  & mae =  1.1505877663287267  & rmse =  1.5087391166380038


Random Forest

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

tree_model = DecisionTreeRegressor()
tree_model.fit(X_train_bow, y_train)
tree_mse = mean_squared_error(y_train, tree_model.predict(X_train_bow))
tree_mae = mean_absolute_error(y_train, tree_model.predict(X_train_bow))
print("Decision Tree training mse = ",tree_mse," & mae = ",tree_mae," & rmse = ", sqrt(tree_mse))

rf_model = RandomForestRegressor(verbose=True, n_jobs = -1)

rf_model.fit(X_train_bow, y_train)
rf_mse = mean_squared_error(y_train, rf_model.predict(X_train_bow))
rf_mae = mean_absolute_error(y_train, rf_model.predict(X_train_bow))

print("Random Forest training mse = ",rf_mse," & mae = ",rf_mae," & rmse = ", sqrt(rf_mse))

Decision Tree training mse =  0.00023410885002873112  & mae =  0.0002089747935995124  & rmse =  0.015300616001610234


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 35.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 78.6min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s


Random Forest training mse =  0.1322844481795036  & mae =  0.2627373806994951  & rmse =  0.36370929075224845


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.8s finished


In [ ]:
#Test
tree_test_mse = mean_squared_error(y_test, tree_model.predict(X_test_bow))
tree_test_mae = mean_absolute_error(y_test, tree_model.predict(X_test_bow))
print(tree_test_mse)
print(tree_test_mae)
rf_test_mse = mean_squared_error(y_test, rf_model.predict(X_test_bow))
rf_test_mae = mean_absolute_error(y_test, rf_model.predict(X_test_bow))
print(rf_test_mse)
print(rf_test_mae)

1.8781527845080406
1.0112080800569496


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.5s


0.9843812018886476
0.7027037569910877


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished


NN (simple)

In [ ]:
print(X_train_bow.shape)
print(X_test_bow.shape)
print(y_train.shape)
print(y_test.shape)
print(type(X_train_bow))

(11160, 33742)
(6323, 33742)
(11160,)
(6323,)
<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
from torch import nn
from torch.utils.data import DataLoader
train_data = []
x, y, xTest, yTest = X_train_bow.toarray(), y_train.tolist(), X_test_bow.toarray(), y_test.tolist()

x = torch.from_numpy(x)
xTest = torch.from_numpy(xTest)
x = x.type(torch.FloatTensor)
xTest = xTest.type(torch.FloatTensor)

#y = torch.from_numpy(y)
y = torch.FloatTensor(y)
trainloader = torch.utils.data.DataLoader([ [x[i], y[i]] for i in range(11160)], shuffle=True, batch_size=32)

In [ ]:
# x = X_train_bow.toarray()
# type(x)
# y = y_train.tolist()

In [ ]:
import torch
from torch.nn.functional import normalize

#https://stackoverflow.com/questions/62372938/understanding-input-shape-to-pytorch-conv1d
class NeuralNetwork(nn.Module):
   def __init__(self):
      super(NeuralNetwork, self).__init__()
      #self.flatten = nn.Flatten(start_dim=1)
      self.loss = nn.MSELoss()
      self.linear_relu_stack = nn.Sequential(
      nn.Linear(33742, 512),
      nn.ReLU(),
      nn.Linear(512, 128),
      nn.ReLU(),
      nn.Linear(128, 32),
      nn.ReLU(),
      nn.Linear(32, 1),
      nn.ReLU()
      )

   def forward(self, x):
      #transpose = x.permute(0,2,1)
      #transpose = self.flatten(transpose)
      #print('x before ', x)
      x = normalize(x, p=2.0)
      #print('x after ', x)
      logits = self.linear_relu_stack(x)
      #vol = normalize(logits, p=2.0)
      return logits

model = NeuralNetwork()
#model.apply(initialize_weights)
model.to(device)

NeuralNetwork(
  (loss): MSELoss()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=33742, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
    (7): ReLU()
  )
)

In [ ]:
#model params
!pip install transformers
import transformers 
from torch.optim import SGD
from transformers import get_scheduler
import torch.nn.functional as F

optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

num_epochs = 1000
num_training_steps = num_epochs * len(trainloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [105]:
import torch 
device = "cuda:0" if torch.cuda.is_available() else "cpu"

#from sklearn.neural_network import MLPClassifier
#model = MLPClassifier(verbose=True)
# model = model.to(device)
#model.fit(X_train_bow, y_train)

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
criterion = nn.MSELoss()
model.train()
losses = []

for epoch in range(num_epochs):
    for batch, y in trainloader:
        outputs = model(batch.to(device))
        loss = criterion(outputs, y.to(device))
        loss = loss.type(torch.FloatTensor)
        loss.backward(retain_graph=True)
      
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    losses.append(loss)
    print(loss)
    print(outputs, y)

from sklearn.metrics import accuracy_score
accuracy_score(y_train, model.predict(X_train_bow))
accuracy_score(y_test, model.predict(X_test_bow))

train_mse = mean_squared_error(y_train, model.predict(X_train_bow))
test_mse = mean_squared_error(y_test, model.predict(X_test_bow))
print(train_mse)
print(test_mse)
train_mae = mean_absolute_error(y_train, model.predict(X_train_bow))
test_mae = mean_absolute_error(y_test, model.predict(X_test_bow))
print(train_mse)
print(test_mse)

  0%|          | 0/349000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([24])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.7251, grad_fn=<ToCopyBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], device='cuda:0', grad_fn=<ReluBackward0>) tensor([-0.8136,  0.1854, -1.0295, -0.6646, -0.5358, -2.1860, -0.0965,  0.1427,
        -0.8005, -0.5039, -0.3129,  0.2910, -0.0523, -0.7752,  1.7968, -0.3778,
        -0.1772,  0.8724,  0.1639, -1.0310,  0.1259,  1.7201, -0.1150, -0.4313])
tensor(0.4881, grad_fn=<ToCopyBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
  

KeyboardInterrupt: ignored

In [111]:
with torch.no_grad():
    
    outputs = model(xTest.to(device))
    print('outputs: ', outputs)
    print(outputs.shape, yTest.shape)
    print('yTest: ', yTest)
    loss = criterion(yTest.to(device), outputs)
    print('loss : ', loss)
    print(loss.shape)
    x = loss
    mae = mean_absolute_error(yTest.cpu(), outputs.cpu())
    print('mae: ', mae)

outputs:  tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], device='cuda:0')
torch.Size([6323, 1]) torch.Size([6323])
yTest:  tensor([-2.7780, -0.3929,  0.6856,  ...,  1.2149, -0.3321,  0.2503])
loss :  tensor(0.9998, device='cuda:0')
torch.Size([])
mae:  0.7044272


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([6323, 1])) that is different to the input size (torch.Size([6323])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Appendix